## A sample proof identifying Lemmas and generating by Derivatives


We illustrate some key features of proving by _learning_ with generative models based on Dependent Type Theory in the [ProvingGround](https://github.com/siddhartha-gadgil/ProvingGround) project. Some details of the project are available on its [website](http://siddhartha-gadgil.github.io/ProvingGround/).

We have a set $M$ with a binary operation $*$, and given elements $e_l$ and $e_m$. We also have terms correspoding to the assumptions

* $e_l * a = e_l$ for all $a\in M$ (left identity axiom),
* $a * e_r = e_r$ for all $a\in M$ (right identity axiom), 
* equality is symmetric and transitive.

From these, we conclude that basic algebraic result that $e_l = e_r$. Note that the proof in its fully expanded form is fairly long, involving the correct instantiations of each of the identity axioms, a use of symmetry of equality and a use of transitivity of equality. In particular it has very low weight in a direct generative model.

However, our system finds the proof quite readily (much less than a minute even running synchronously in the notebook), due to a few key principles.

* We use function application with _unification_, thus we can apply symmetry of equality directly to a term of type $a = b$ even though the type of symmetry is $\Pi_{x : M}\Pi_{y: M} x = y \to y= x$ as $x=a$ and $y=b$ are inferred.
* Most importantly, we learn that a lemma such as $e_l = e_l * e_r$, which is obtained by instantiation __and__ symmetry, is non-trivial (while only instantiation is not) based on a cost combining entropy of the initial distribution with the Kullback-Liebler divergence of _theorems_ (inhabited types) from _their proofs_.
* We also gain efficiency by using the _derivatives_ of the generative models to find consequences of lemmas.


We import the locally published  core of ProvingGround.

In [1]:
import $ivy.`io.github.siddhartha-gadgil::provingground-core-jvm:0.1.1-SNAPSHOT`

import $ivy.$                                                                   

We import the relevant packages, including the one with our example.

In [2]:
import provingground._ , interface._, HoTT._
import learning._
import library._, MonoidSimple._

import provingground._ , interface._, HoTT._

import learning._

import library._, MonoidSimple._

In [3]:
 repl.pprinter.bind(translation.FansiShow.fansiPrint)

To set up a _generator_. For simplicity in this demo, we are not generating $\lambda$'s and $\Pi$-types

In [4]:
val tg = TermGenParams(lmW=0, piW=0)

tg: TermGenParams = TermGenParams(
  0.1,
  0.1,
  0.1,
  0.0,
  0.0,
  0.05,
  0.05,
  0.05,
  0.0,
  0.0,
  0.0,
  0.0,
  0.3,
  0.7,
  0.5,
  0.0,
  0.0,
  0.0
)

We set up the initial state for learning.

In [5]:
val ts = TermState(dist1, dist1.map(_.typ))

ts: TermState = TermState(
  FiniteDistribution(
    Vector(
      Weighted(e_l, 0.047619047619047616),
      Weighted(e_r, 0.047619047619047616),
      Weighted(mul, 0.047619047619047616),
      Weighted(eqM, 0.047619047619047616),
      Weighted(axiom_{eqM(a)(a)}, 0.047619047619047616),
      Weighted(axiom_{(eqM(a)(b) \to eqM(b)(a))}, 0.047619047619047616),
      Weighted(axiom_{(eqM(a)(b) \to (eqM(b)(c) \to eqM(a)(c)))}, 0.047619047619047616),
      Weighted(axiom_{eqM(mul(e_l)(a))(a)}, 0.047619047619047616),
      Weighted(axiom_{eqM(mul(a)(e_r))(a)}, 0.047619047619047616),
      Weighted(eqM, 0.2857142857142857),
      Weighted(mul, 0.2857142857142857)
    )
  ),
  FiniteDistribution(
    Vector(
      Weighted(M, 0.047619047619047616),
      Weighted(M, 0.047619047619047616),
      Weighted((M → (M → M)), 0.047619047619047616),
      Weighted((M → (M → 𝒰 )), 0.047619047619047616),
      Weighted(∏(a : M){ eqM(a)(a) }, 0.047619047619047616),
      Weighted(∏(a : M){ ∏(b : M){ (eq

We use the _monix_ library for concurrency, though in this notebook we run code synchronously.

In [6]:
import monix.execution.Scheduler.Implicits.global 

import monix.execution.Scheduler.Implicits.global 

We synchronously evolve to the next state with our generative model.

In [7]:
val evT = tg.evolvedStateTask(ts, 0.00003)

evT: monix.eval.Task[EvolvedState] = Map(
  FlatMap(
    FlatMap(Async(<function2>, true, true, true), monix.eval.Task$$Lambda$2531/1439912625@6354110f),
    provingground.learning.TermGenParams$$Lambda$2532/1405125416@4c2484e0
  ),
  provingground.learning.TermGenParams$$Lambda$2533/1232311273@7f29f6f5,
  0
)

In [8]:
val ev = evT.runSyncUnsafe() 

ev: EvolvedState = EvolvedState(
  TermState(
    FiniteDistribution(
      Vector(
        Weighted(e_l, 0.047619047619047616),
        Weighted(e_r, 0.047619047619047616),
        Weighted(mul, 0.047619047619047616),
        Weighted(eqM, 0.047619047619047616),
        Weighted(axiom_{eqM(a)(a)}, 0.047619047619047616),
        Weighted(axiom_{(eqM(a)(b) \to eqM(b)(a))}, 0.047619047619047616),
        Weighted(axiom_{(eqM(a)(b) \to (eqM(b)(c) \to eqM(a)(c)))}, 0.047619047619047616),
        Weighted(axiom_{eqM(mul(e_l)(a))(a)}, 0.047619047619047616),
        Weighted(axiom_{eqM(mul(a)(e_r))(a)}, 0.047619047619047616),
        Weighted(eqM, 0.2857142857142857),
        Weighted(mul, 0.2857142857142857)
      )
    ),
    FiniteDistribution(
      Vector(
        Weighted(M, 0.047619047619047616),
        Weighted(M, 0.047619047619047616),
        Weighted((M → (M → M)), 0.047619047619047616),
        Weighted((M → (M → 𝒰 )), 0.047619047619047616),
        Weighted(∏(a : M){ eqM(a)(a) }

### Identifying Lemmas

We identify lemmas as types that we know to be inhabited but are not inhabited by terms in the generating set. We have kept things simple, so there are fewer lemmas to start with. But the main step is _learning_ the weights of the lemmas.

The evolution of weights is generated as an _iterator_. As this is a structure that is generated just once, we store it as a _stream_, which is a lazy, infinite list.

In [9]:
import EntropyAtomWeight._ 
val its = evolvedLemmaIters(ev, 1, 1)

import EntropyAtomWeight._ 

its: Vector[(Typ[Term], Iterator[Double])] = Vector(
  (eqM(e_r)(e_r), <iterator>),
  (eqM(mul(e_l)(e_r))(e_l), <iterator>),
  (eqM(e_l)(e_l), <iterator>),
  (eqM(mul(e_l)(e_l))(e_l), <iterator>),
  (eqM(e_l)(mul(e_l)(e_r)), <iterator>),
  (eqM(e_l)(mul(e_l)(e_l)), <iterator>),
  (eqM(mul(e_l)(e_r))(e_r), <iterator>),
  (eqM(mul(e_r)(e_r))(e_r), <iterator>),
  (eqM(e_r)(mul(e_l)(e_r)), <iterator>),
  (eqM(e_r)(mul(e_r)(e_r)), <iterator>)
)

In [10]:
val streams = its.map{case (tp, it) => (tp, it.toStream)} 

streams: Vector[(Typ[Term], Stream[Double])] = Vector(
  (eqM(e_r)(e_r), [0.007180424143126931 ...),
  (eqM(mul(e_l)(e_r))(e_l), [0.002653613511816296 ...),
  (eqM(e_l)(e_l), [0.007180424143126931 ...),
  (eqM(mul(e_l)(e_l))(e_l), [0.002653613511816296 ...),
  (eqM(e_l)(mul(e_l)(e_r)), [1.8739154055186634E-4 ...),
  (eqM(e_l)(mul(e_l)(e_l)), [1.8739154055186634E-4 ...),
  (eqM(mul(e_l)(e_r))(e_r), [0.002653613511816296 ...),
  (eqM(mul(e_r)(e_r))(e_r), [0.002653613511816296 ...),
  (eqM(e_r)(mul(e_l)(e_r)), [1.8739154055186634E-4 ...),
  (eqM(e_r)(mul(e_r)(e_r)), [1.8739154055186634E-4 ...)
)

We see three snapshots of the evolution of weights, at times $0$, $1000$ and $10000$. We also plot the evolution below.

In [11]:
streams.map{case (tp, s) => (tp, s(0), s(1000), s(10000))}

res10: Vector[(Typ[Term], Double, Double, Double)] = Vector(
  (eqM(e_r)(e_r), 0.007180424143126931, 3.149476255819165E-4, 1.851907551274217E-5),
  (eqM(mul(e_l)(e_r))(e_l), 0.002653613511816296, 1.4782068526778965E-4, 1.1469129744309385E-5),
  (eqM(e_l)(e_l), 0.007180424143126931, 3.149476255819165E-4, 1.851907551274217E-5),
  (eqM(mul(e_l)(e_l))(e_l), 0.002653613511816296, 1.4782068526778965E-4, 1.1469129744309385E-5),
  (eqM(e_l)(mul(e_l)(e_r)), 1.8739154055186634E-4, 0.002470320793641852, 0.0024776533663946215),
  (eqM(e_l)(mul(e_l)(e_l)), 1.8739154055186634E-4, 0.002470320793641852, 0.0024776533663946215),
  (eqM(mul(e_l)(e_r))(e_r), 0.002653613511816296, 1.4782068526778965E-4, 1.1469129744309385E-5),
  (eqM(mul(e_r)(e_r))(e_r), 0.002653613511816296, 1.4782068526778965E-4, 1.1469129744309385E-5),
  (eqM(e_r)(mul(e_l)(e_r)), 1.8739154055186634E-4, 0.002470320793641852, 0.0024776533663946215),
  (eqM(e_r)(mul(e_r)(e_r)), 1.8739154055186634E-4, 0.002470320793641852, 0.002477653366394

In [12]:
import $ivy.`org.plotly-scala::plotly-almond:0.5.2`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._

import $ivy.$                                      


import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

In [13]:
val scatters = streams.map{case (typ, s) => Scatter((1 to 50).map(i => (i * 50)), (1 to 50).map(i => s(i * 50)), name=typ.toString )}
val tsdata = Seq(scatters: _*)

scatters: Vector[Scatter] = Vector(
  Scatter(
    Some(
      Doubles(
        Vector(
          50.0,
          100.0,
          150.0,
          200.0,
          250.0,
          300.0,
          350.0,
          400.0,
          450.0,
          500.0,
          550.0,
          600.0,
          650.0,
          700.0,
          750.0,
          800.0,
          850.0,
          900.0,
          950.0,
          1000.0,
          1050.0,
          1100.0,
          1150.0,
          1200.0,
          1250.0,
          1300.0,
          1350.0,
          1400.0,
          1450.0,
          1500.0,
          1550.0,
          1600.0,
          1650.0,
          1700.0,
          1750.0,
          1800.0,
          1850.0,
          1900.0,
          1950.0,
          2000.0,
          2050.0,
          2100.0,
          2150.0,
          2200.0,
...
tsdata: Seq[Scatter] = List(
  Scatter(
    Some(
      Doubles(
        Vector(
          50.0,
          100.0,
          150.0,
     

### Plot of the evolution

res13: String = plot-1962139029

In [14]:
tsdata.plot(title="Evolution of the lemma coefficients")

We pick the lemma that will give us the result. Note that there are four lemmas that have high weight after the evolution, two of which will work, so one could evolve along all of them (which we do in the interactive code).

In [15]:
val pf = "lemma" :: toTyp(streams(8)._1)
pf.typ

pf: U = lemma
res14_1: Typ[U] = eqM(e_r)(mul(e_l)(e_r))

### Using the lemma

For quick further exploration, we use the _derivatives_ of the recursive generating rules in the direction of the new lemma.

In [16]:
val tt = tg.nextTangStateTask(ev.result, TermState(FiniteDistribution.unif(pf), FiniteDistribution.empty), math.pow(10.0, -4)) 

tt: monix.eval.Task[TermState] = FlatMap(
  FlatMap(Async(<function2>, true, true, true), monix.eval.Task$$Lambda$2531/1439912625@14bf3c70),
  provingground.learning.TermGenParams$$Lambda$3285/1548968450@2cc08640
)

In [17]:
val tts = tt.runSyncUnsafe()

tts: TermState = TermState(
  FiniteDistribution(
    Vector(
      Weighted((@a : M) ↦ (@b : M) ↦ @b, 0.0036008547216669308),
      Weighted((@a : M) ↦ (@b : M) ↦ @a, 0.002520598305166852),
      Weighted(
        axiom_{(eqM(a)(b) \to (eqM(b)(c) \to eqM(a)(c)))}(e_r)(e_r)(mul(e_l)(e_l))(axiom_{(eqM(a)(b) \to (eqM(b)(c) \to eqM(a)(c)))}(e_r)(mul(e_l)(e_r))(e_r)(lemma)(axiom_{eqM(mul(e_l)(a))(a)}(e_r))),
        3.3661218078100615E-6
      ),
      Weighted((@a : M) ↦ mul(@a), 0.018254128413825316),
      Weighted(
        axiom_{(eqM(a)(b) \to eqM(b)(a))}(e_r)(mul(e_l)(e_r))(axiom_{(eqM(a)(b) \to eqM(b)(a))}(mul(e_l)(e_r))(e_r)(axiom_{(eqM(a)(b) \to eqM(b)(a))}(e_r)(mul(e_l)(e_r))(lemma))),
        6.961385296652488E-4
      ),
      Weighted(
        ($brpwn : eqM(mul(e_r)(e_l))($brpvj)) ↦ axiom_{(eqM(a)(b) \to (eqM(b)(c) \to eqM(a)(c)))}(mul(e_r)(e_l))(e_r)(mul(e_l)(e_r))($brpwn)(lemma),
        4.8791369447440146E-4
      ),
      Weighted(lemma, 0.6512488371186822),
      Weighted

## The Proof

The next step indeed gives a proof of the desired result $e_l = e_r$. Note that this has a high weight of 1.2%

In [18]:
tts.terms.filter(_.typ == eqM(l)(r)) 

res17: FiniteDistribution[Term] = FiniteDistribution(
  Vector(
    Weighted(
      axiom_{(eqM(a)(b) \to eqM(b)(a))}(e_r)(e_l)(axiom_{(eqM(a)(b) \to (eqM(b)(c) \to eqM(a)(c)))}(e_r)(mul(e_l)(e_r))(e_l)(lemma)(axiom_{eqM(mul(a)(e_r))(a)}(e_l))),
      1.0099663101516774E-4
    )
  )
)

### Other attempts

In [19]:
val ss = streams.filter{case (tp, s) => s(10000) > s(1000)}.map{case (tp, s) => (tp, s(10000))}

ss: Vector[(Typ[Term], Double)] = Vector(
  (eqM(e_l)(mul(e_l)(e_r)), 0.0024776533663946215),
  (eqM(e_l)(mul(e_l)(e_l)), 0.0024776533663946215),
  (eqM(e_r)(mul(e_l)(e_r)), 0.0024776533663946215),
  (eqM(e_r)(mul(e_r)(e_r)), 0.0024776533663946215)
)

In [20]:
val ls = FiniteDistribution(ss.map{case (tp, p) => Weighted("lemma" :: tp, p * 10)})

ls: FiniteDistribution[Term] = FiniteDistribution(
  Vector(
    Weighted(lemma, 0.024776533663946214),
    Weighted(lemma, 0.024776533663946214),
    Weighted(lemma, 0.024776533663946214),
    Weighted(lemma, 0.024776533663946214)
  )
)

In [21]:
val ts1 = TermState((dist1 ++ ls).normalized(), dist1.map(_.typ))

ts1: TermState = TermState(
  FiniteDistribution(
    Vector(
      Weighted(e_l, 0.043325249598357325),
      Weighted(e_r, 0.043325249598357325),
      Weighted(mul, 0.043325249598357325),
      Weighted(eqM, 0.043325249598357325),
      Weighted(axiom_{eqM(a)(a)}, 0.043325249598357325),
      Weighted(axiom_{(eqM(a)(b) \to eqM(b)(a))}, 0.043325249598357325),
      Weighted(axiom_{(eqM(a)(b) \to (eqM(b)(c) \to eqM(a)(c)))}, 0.043325249598357325),
      Weighted(axiom_{eqM(mul(e_l)(a))(a)}, 0.043325249598357325),
      Weighted(axiom_{eqM(mul(a)(e_r))(a)}, 0.043325249598357325),
      Weighted(eqM, 0.25995149759014397),
      Weighted(mul, 0.25995149759014397),
      Weighted(lemma, 0.022542439608624024),
      Weighted(lemma, 0.022542439608624024),
      Weighted(lemma, 0.022542439608624024),
      Weighted(lemma, 0.022542439608624024)
    )
  ),
  FiniteDistribution(
    Vector(
      Weighted(M, 0.047619047619047616),
      Weighted(M, 0.047619047619047616),
      Weighted((M → (M 

In [22]:
val evT1 = tg.evolvedStateTask(ts1, 0.000003)

evT1: monix.eval.Task[EvolvedState] = Map(
  FlatMap(
    FlatMap(Async(<function2>, true, true, true), monix.eval.Task$$Lambda$2531/1439912625@10772d94),
    provingground.learning.TermGenParams$$Lambda$2532/1405125416@75d438db
  ),
  provingground.learning.TermGenParams$$Lambda$2533/1232311273@209812ec,
  0
)

In [23]:
val ev1 = evT1.runSyncUnsafe()

ev1: EvolvedState = EvolvedState(
  TermState(
    FiniteDistribution(
      Vector(
        Weighted(e_l, 0.043325249598357325),
        Weighted(e_r, 0.043325249598357325),
        Weighted(mul, 0.043325249598357325),
        Weighted(eqM, 0.043325249598357325),
        Weighted(axiom_{eqM(a)(a)}, 0.043325249598357325),
        Weighted(axiom_{(eqM(a)(b) \to eqM(b)(a))}, 0.043325249598357325),
        Weighted(axiom_{(eqM(a)(b) \to (eqM(b)(c) \to eqM(a)(c)))}, 0.043325249598357325),
        Weighted(axiom_{eqM(mul(e_l)(a))(a)}, 0.043325249598357325),
        Weighted(axiom_{eqM(mul(a)(e_r))(a)}, 0.043325249598357325),
        Weighted(eqM, 0.25995149759014397),
        Weighted(mul, 0.25995149759014397),
        Weighted(lemma, 0.022542439608624024),
        Weighted(lemma, 0.022542439608624024),
        Weighted(lemma, 0.022542439608624024),
        Weighted(lemma, 0.022542439608624024)
      )
    ),
    FiniteDistribution(
      Vector(
        Weighted(M, 0.047619047619047616),
 

In [24]:
ev1.result.terms.filter(_.typ == eqM(l)(r)) 

res23: FiniteDistribution[Term] = FiniteDistribution(Vector())

In [25]:
ev1.result.terms.map(_.typ).entropyVec

res24: Vector[Weighted[Typ[U] forSome { type U >: x$1 <: Term with Subs[U]; val x$1: Term }]] = Vector(
  Weighted((M → (M → M)), 2.108194369089061),
  Weighted((M → (M → 𝒰 )), 2.108194369089061),
  Weighted(M, 3.4151793068089265),
  Weighted((M → 𝒰 ), 4.156465755821471),
  Weighted((M → M), 4.156808506457434),
  Weighted(∏(a : M){ eqM(mul(e_l)(a))(a) }, 4.915549291146665),
  Weighted(
    ∏(a : M){ ∏(b : M){ ∏(c : M){ (eqM(a)(b) → (eqM(b)(c) → eqM(a)(c))) } } },
    4.915549291146665
  ),
  Weighted(∏(a : M){ ∏(b : M){ (eqM(a)(b) → eqM(b)(a)) } }, 4.915549291146665),
  Weighted(∏(a : M){ eqM(a)(a) }, 4.915549291146665),
  Weighted(∏(@a : M){ eqM(mul(@a)(e_r))(@a) }, 4.915549291146665),
  Weighted(𝒰 , 5.183963491769294),
  Weighted(eqM(e_l)(mul(e_l)(e_r)), 5.931128018615346),
  Weighted(eqM(e_l)(mul(e_l)(e_l)), 5.931128018615346),
  Weighted(eqM(e_r)(mul(e_l)(e_r)), 5.931128018615346),
  Weighted(eqM(e_r)(mul(e_r)(e_r)), 5.931128018615346),
  Weighted(eqM(e_r)(e_r), 7.403066436148555),

In [26]:
import Unify._

import Unify._

In [27]:
unifApply(trans, "lemma" :: ss(0)._1, Vector())

res26: Option[Term] = None

In [28]:
ss(0)._1
trans.typ

res27_0: Typ[Term] = eqM(e_l)(mul(e_l)(e_r))
res27_1: Typ[FuncLike[Term, FuncLike[Term, FuncLike[Term, Func[Term, Func[Term, Term]]]]]] = ∏(a : M){ ∏(b : M){ ∏(c : M){ (eqM(a)(b) → (eqM(b)(c) → eqM(a)(c))) } } }

In [29]:
unifApply(sym, "lemma" :: ss(0)._1, Vector())

res28: Option[Term] = None

In [30]:
import Fold._
domain(trans)

import Fold._

res29_1: Typ[Term] = M

In [31]:
val arg = "lemma" :: ss(0)._1

arg: Term = lemma

In [32]:
unify(M, arg.typ, _ => false)

res31: Option[Map[Term, Term]] = None

In [33]:
arg.typ

res32: Typ[U] = eqM(e_l)(mul(e_l)(e_r))

In [34]:
appln(sym, arg)

res33: Option[Term] = Some(sym(e_l)(mul(e_l)(e_r))(lemma))

In [35]:
appln(trans, arg)

res34: Option[Term] = Some(($awqpot : M) ↦ trans(e_l)(mul(e_l)(e_r))($awqpot)(lemma))

In [36]:
val fn = appln(trans, arg).get
fn.typ

fn: Term = ($awqpvj : M) ↦ trans(e_l)(mul(e_l)(e_r))($awqpvj)(lemma)
res35_1: Typ[U] = ∏($awqpvj : M){ (eqM(mul(e_l)(e_r))($awqpvj) → eqM(e_l)($awqpvj)) }